In [1]:
import requests
import pandas as pd
import re
from ast import literal_eval
from lxml import html
import tqdm
import time

#### 1. Yöntem. Datayı ana kaynaktan alır ve içerikler tam olarak çekilir.

In [99]:
# Site bizi bot sanmasın diye headersları da belirtip istek atıyoruz. Headersız istediğimiz datayı sunmuyor. Postman' e cURL import edilerek ilgili script elde edilir.

url = "https://www.sikayetvar.com/agesa-hayat-ve-emeklilik?page={0}".format(1)

headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="97", "Chromium";v="97"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Referer': 'https://www.sikayetvar.com/',
    'Accept-Language': 'en-US,en;q=0.9,de;q=0.8,ms;q=0.7,tr;q=0.6,ru;q=0.5,az;q=0.4',
    'Cookie': '_svhash=cf0a6e6adbab352a58b4c4c49425e701; _gid=GA1.2.1338087073.1643790897; sv-cookie-msg=true; _svuid=s%3ArC54L_-rT2GS1ncuDhAguytVZtN1TOVe.N40Sv4mS9L24CHcpBxc1m8Ln6NApkA0UoJrDGqtcco8; JSESSIONID=709A7F267AE6530B30707F8E9C0CE7F2; _ga=GA  .1067635138.1642427186; _ga_FXTD430HRD=GS1.1.1643958543.5.1.1643958957.44; resolved-complaint-count={%22current%22:1875553%2C%22target%22:1875568}'
      }

response = requests.request("GET", url, headers=headers)

In [100]:
# html.fromstring provides us an object of HtmlElement type. This object has the xpath method which we can use to query the HTML document.This provides us with a structured way to extract information from an HTML document.
tree = html.fromstring(response.text)
tree

<Element html at 0x7f5ca7a42868>

In [101]:
# Reviewler site source code içerisinde iki <script type=application/ld+json> arasında yer alıyor
temp_list = tree.xpath("//script[@type='application/ld+json']")
temp_list

[<Element script at 0x7f5ca7a4ef48>, <Element script at 0x7f5caee42548>]

In [ ]:
# Baştaki ve sondaki boşlukları silmek için, boşluk olunca literal_eval hata alıyor
raw_json = temp_list[0].text.strip()

In [103]:
# literal_aval() function helps to make strings more readable , and also helps python to understand the datatype. String halındekı json dosyasını gercek json'a çeviriyor.
raw_data = literal_eval(raw_json)
raw_data

{'@context': 'https://schema.org',
 '@type': 'WebPage',
 'inLanguage': 'tr-TR',
 'isFamilyFriendly': 'true',
 'url': 'https://www.sikayetvar.com/agesa-hayat-ve-emeklilik',
 'headline': 'AgeSA Hayat ve Emeklilik',
 'description': "AvivaSA Emeklilik şikayet ve yorumları Şikayetvar'da! Siz de AvivaSA Emeklilik şikayet ve yorumlarınızı sayfamız üzerinden yayınlayabilirsiniz.",
 'relatedLink': ['https://www.sikayetvar.com/agesa-hayat-ve-emeklilik/iptal',
  'https://www.sikayetvar.com/agesa-hayat-ve-emeklilik/para-iadesi',
  'https://www.sikayetvar.com/agesa-hayat-ve-emeklilik/musteri-hizmetleri',
  'https://www.sikayetvar.com/agesa-hayat-ve-emeklilik/telefon-numarasi',
  'https://www.sikayetvar.com/agesa-hayat-ve-emeklilik/para-cekme',
  'https://www.sikayetvar.com/agesa-hayat-ve-emeklilik/police-iptali',
  'https://www.sikayetvar.com/agesa-hayat-ve-emeklilik/fon',
  'https://www.sikayetvar.com/agesa-hayat-ve-emeklilik/odeme',
  'https://www.sikayetvar.com/agesa-hayat-ve-emeklilik/iptal-etm

In [104]:
# Review başlığı altında toplanan veriler
raw_data['@graph']['review'][7]

{'@type': 'Review',
 'headline': 'Agesa Hayat Ve Emeklilik\xa0\xa0Aktaracağız Deyip Telefonu Kapatıyorlar',
 'itemReviewed': {'@type': 'Thing', 'name': 'AgeSA Hayat ve Emeklilik'},
 'reviewRating': {'@type': 'Rating',
  'bestRating': 5,
  'worstRating': 1,
  'ratingValue': 1},
 'author': {'@type': 'Person',
  'name': 'Es Ka',
  'url': 'https://www.sikayetvar.com/uye/es-ka-5401162'},
 'datePublished': '2022-02-05',
 'reviewBody': 'İşsizlik teminatlı kredi çekmiştim. 8 gündür arıyorum. Aktaracağız deyip telefonu kapatıyorlar. Üstelik defalarca mail attım olayı anlattım ilgili birime ulaşamıyorum beni arayın, 3 kere talep oluşturdum geri aranmak için hala bir haber yok. İlgili birim yerine memnuniyet için arıyorlar beni şaka gibi dalga geçiyorlar başka açıklaması yok. 0 puan vermişim diye arıyor. Onlara da durumu anlatıyorum arıcaklar diyorlar. Daha kötü bir sigorta olamaz.',
 'publisher': {'@type': 'Organization',
  'name': 'Şikayetvar',
  'sameAs': 'https://www.sikayetvar.com',
  'logo'

In [105]:
# See an example's values in the 7.index
raw_data['@graph']['review'][7]['headline']

'Agesa Hayat Ve Emeklilik\xa0\xa0Aktaracağız Deyip Telefonu Kapatıyorlar'

In [106]:
raw_data['@graph']['review'][7]['datePublished']

'2022-02-05'

In [107]:
raw_data['@graph']['review'][7]['reviewRating']['ratingValue']

1

In [108]:
raw_data['@graph']['review'][7]['reviewBody']

'İşsizlik teminatlı kredi çekmiştim. 8 gündür arıyorum. Aktaracağız deyip telefonu kapatıyorlar. Üstelik defalarca mail attım olayı anlattım ilgili birime ulaşamıyorum beni arayın, 3 kere talep oluşturdum geri aranmak için hala bir haber yok. İlgili birim yerine memnuniyet için arıyorlar beni şaka gibi dalga geçiyorlar başka açıklaması yok. 0 puan vermişim diye arıyor. Onlara da durumu anlatıyorum arıcaklar diyorlar. Daha kötü bir sigorta olamaz.'

In [67]:
# Find the max page number of reviews
page_list = tree.xpath("//ul[@class='pagination']/li/a/text()")      # contains the numbers that seen in the website
page_list = [int(i) for i in page_list if i != ' ' ]
max(page_list)

193

In [68]:
headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="97", "Chromium";v="97"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Referer': 'https://www.sikayetvar.com/',
    'Accept-Language': 'en-US,en;q=0.9,de;q=0.8,ms;q=0.7,tr;q=0.6,ru;q=0.5,az;q=0.4',
    'Cookie': '_svhash=cf0a6e6adbab352a58b4c4c49425e701; _gid=GA1.2.1338087073.1643790897; sv-cookie-msg=true; _svuid=s%3ArC54L_-rT2GS1ncuDhAguytVZtN1TOVe.N40Sv4mS9L24CHcpBxc1m8Ln6NApkA0UoJrDGqtcco8; JSESSIONID=709A7F267AE6530B30707F8E9C0CE7F2; _ga=GA  .1067635138.1642427186; _ga_FXTD430HRD=GS1.1.1643958543.5.1.1643958957.44; resolved-complaint-count={%22current%22:1875553%2C%22target%22:1875568}'
}


df1 = pd.DataFrame( [], columns = ["Company", "Headline", "DatePublished" , "RatingValue", "Review"] )    # for reviews that has rating value
df2 = pd.DataFrame( [], columns = ["Company", "Headline", "DatePublished" , "RatingValue", "Review"] )   # for reviews that has'nt rating value
for page in tqdm.tqdm(range(1, max(page_list)+1) ):
    url = "https://www.sikayetvar.com/agesa-hayat-ve-emeklilik?page={0}".format(str(page))
    response = requests.request("GET", url, headers=headers)
    tree = html.fromstring(response.text)
    temp_list = tree.xpath("//script[@type='application/ld+json']")
    raw_json = temp_list[0].text.strip()
    raw_data = literal_eval(raw_json)

    for i in range(0,len(raw_data['@graph']['review'])):
        try:
            rev = ['Agesa Hayat Ve Emeklilik',raw_data['@graph']['review'][i]['headline'],raw_data['@graph']['review'][i]['datePublished'],raw_data['@graph']['review'][i]['reviewRating']['ratingValue'],raw_data['@graph']['review'][i]['reviewBody'] ]
            df1_length = len(df1)
            df1.loc[df1_length] = rev

        except KeyError:
            rev2 = ['Agesa Hayat Ve Emeklilik',raw_data['@graph']['review'][i]['headline'],raw_data['@graph']['review'][i]['datePublished'],'-',raw_data['@graph']['review'][i]['reviewBody'] ]
            df2_length = len(df2)
            df2.loc[df2_length] = rev2

        except  Exception:
            pass

    time.sleep(0.5)     # to set time between loops

df = pd.concat([df1,df2])
agesa_df = df.copy()
agesa_df.reset_index(drop=True, inplace=True)

100%|██████████| 193/193 [03:03<00:00,  1.05it/s]


In [75]:
agesa_df

,Company,Headline,DatePublished,RatingValue,Review
0,Agesa Hayat Ve Emeklilik,Agesa Hayat Ve Emeklilik Aktaracağız Deyip Te...,2022-02-05,1,İşsizlik teminatlı kredi çekmiştim. 8 gündür a...
1,Agesa Hayat Ve Emeklilik,Agesa Hayat Ve Emeklilik BES'in Tamamını İade ...,2022-02-05,1,Otomatik BES katılımı gerçekleştirildiği için ...
2,Agesa Hayat Ve Emeklilik,AgeSA Hayat ve Emeklilik Sigorta İptali,2022-02-04,1,2021 Mayıs ayında Akbank'tan kullanmış olduğum...
3,Agesa Hayat Ve Emeklilik,Agesa Hayat ve Emeklilik Hayat Sigortası İptali,2022-02-03,1,Akbank'tan kullanmış olduğum 42.000₺ kredi içi...
4,Agesa Hayat Ve Emeklilik,Agesa Hayat Ve Emeklilik Mobil 300 TL Kesint...,2022-02-02,5,Akbank Agesa işlemleri herhangi yatırım yapmad...
...,...,...,...,...,...
4621,Agesa Hayat Ve Emeklilik,Avivasa Emeklilik BES İçin Muhatap Bulamıyorum!,2017-02-07,-,"İletişim merkezine ulaşamıyorum, çözümcül bir ..."
4622,Agesa Hayat Ve Emeklilik,Avivasa Emeklilik Otomatik Katılımdan Çıkamıyo...,2017-02-07,-,Otomatik katılım ile Avivasa'dan BES sözleşmem...
4623,Agesa Hayat Ve Emeklilik,Avivasa Emeklilik 'e Ulaşamıyorum,2017-02-07,-,Devletin zorunlu yaptığı bireysel emekliliğin ...
4624,Agesa Hayat Ve Emeklilik,Avivasa Emeklilik Hayat Sigortası İptali Sorunu!,2017-02-07,-,Akbank A.Ş'den konut kredisi çektim ancak tara...


## 2.Yöntem
##### Bu yöntemde sadece ekranda görünen bilgileri alabiliyorsun. yorumda devamını oku kısmındaki yorumlar gelmez örneğin.
##### Path'leri almak için; review için örneğin ; site Console üzerinde   $x("normalize-space(//*[@id='main-content']/div[1]/div[2]/article[1]/section/p)")   yazılarak erişilebiliyor.

In [299]:
# !pip install scrapy
import scrapy

In [298]:
name_xpath = "normalize-space(//*[@id='main-content']/div[1]/div[2]/article[1]/header/div/div/a/span)"
name = scrapy.Selector(text = response.text).xpath(name_xpath).extract()[0]
name

'Özlem'

In [302]:
review_xpath = "normalize-space(//*[@id='main-content']/div[1]/div[2]/article[1]/section/p)"
review = scrapy.Selector(text = response.text).xpath(review_xpath).extract()[0]
review

'Otomatik katılım ile Avivasa\'dan BES sözleşmem yapıldı. Cayma hakkı için Avivasa\'nın 1tıklaotomatikkatilim web sitesine TCK ile giriş yapılıyor. Sisteme giriş yapıldığı anda "Kişisel Verilerin işlenmesi ve 3. kişilerle paylaşımı izin bilgilendirme formu" bizi karşılıyor. Buna "tamam" demeden (ret şa...'

In [305]:
headline_xpath = "normalize-space(//*[@id='main-content']/div[1]/div[2]/article[1]/section/h2/a)"
headline = scrapy.Selector(text = response.text).xpath(headline_xpath).extract()[0]
headline

'Avivasa Emeklilik Otomatik Katılımdan Çıkamıyorum!'

In [313]:
for i in range(1,20):
    headline_xpath = "normalize-space(//*[@id='main-content']/div[1]/div[2]/article[{0}]/section/h2/a)".format(i)
    name_xpath = "normalize-space(//*[@id='main-content']/div[1]/div[2]/article[{0}]/header/div/div/a/span)".format(i)
    review_xpath = "normalize-space(//*[@id='main-content']/div[1]/div[2]/article[{0}]/section/p)".format(i)
    headline = scrapy.Selector(text = response.text).xpath(headline_xpath).extract()[0]
    name = scrapy.Selector(text = response.text).xpath(name_xpath).extract()[0]
    review = scrapy.Selector(text = response.text).xpath(review_xpath).extract()[0]
    print(name,'\n', headline,'\n', review,'\n')

Burak 
 Agesa Hayat ve Emeklilik Hayat Sigortası İptali 
 Akbank'tan kullanmış olduğum 42.000₺ kredi için benden bana sorulmadan 1972,06₺ hayat sigortası için para kesildi. En kısa sürede hayat sigortasının iptalini ve tutarın tarafıma iadesini istiyorum. Aksi takdirde Tüketici Hakem Heyeti'ne başvuracağım. 

Enver 
 Agesa Hayat Ve Emeklilik Cayma Bedeli Kesintisi! 
 Agesa sigorta cayma sonrası paranın 4 te 3 yiyor 40 günlük bir sigorta için ortalama 750 TL kesinti yapmış olacak gibi değil insanları ya sigorta da kalın ya da cayma yok demeye getiriyor 40 zaman diliminde bir şey olsa haklılar fakat bu kadar kesinti kabul edilir gibi değil el insaf. 

Merve 
 Agesa Hayat Ve Emeklilik  Müşteri Temsilcisine Bağlanamıyorum! 
 444 11 11 aradım müşteri temsilcisi bağlandım bilgilerimi verdim BES iptali için sizi başkasına aktarıyoruz diyor. 30 dakika bekledim kimse yanıt vermedi lütfen tarafıma dönüş yapılmasını rica ediyorum.-444 11 11 aradım müşteri temsilcisi bağlandım bilgilerimi verdim B